__NOTE: Do NOT 'Run All' on this notebook.__
- Edit the variables in the cell below.

In [ ]:
#You _will_ need to edit this variable
jobName='zwx-debra'

#You may need to edit these (if you know what you're doing.)
config={
    'enableBucket':'false',
    'maxResolution':'512,512',
    'xformers':'false',
    'minStepsImage':'200',  #Recommended: 100-200
    'minStepsEpoch':'2800',   #Normally around 1500. Recommended: minStepsImage * # of images
    'numEpochs':'10'
}

- Continue running the next two cells.

In [4]:
imageDir='/workspace/training'
configFile='/workspace/runpod_ai/scripts/LoraBasicSettings.json'

In [6]:
!mkdir -p $imageDir
!mkdir -p $imageDir/logs
!mkdir -p $imageDir/images/tmp
!mkdir -p $imageDir/output
!mkdir -p $imageDir/reg
print('Upload training pictues to '+imageDir+'/images/tmp')

Upload training pictues to /workspace/training/images/tmp


In [7]:
from IPython.display import display
import ipywidgets as widgets
from pathlib import Path

upload_directory = imageDir+'/images/tmp'  # Specify the directory to upload the files

uploaded = widgets.FileUpload(accept='', multiple=True)

def upload_files(b):
    for filename in uploaded.value:
        content = uploaded.value[filename]['content']
        file_path = Path(upload_directory) / filename
        with open(file_path, "wb") as f:
            f.write(content)
        print(f"Uploaded file: {file_path}")

upload_button = widgets.Button(description='Upload Files')
upload_button.on_click(upload_files)
display(upload_button)
display(uploaded)


Button(description='Upload Files', style=ButtonStyle())

FileUpload(value=(), description='Upload', multiple=True)

__Pause here__
- Upload your training images to the directory named above.
- When all pictures are uploaded, run the next cell.

In [ ]:
#Calculate steps/image, rename image directory
import os

directory = imageDir+'/images/tmp'  # Specify the directory path
numFiles = len([f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))])

# min 100/image, min 1500/epoch
numSteps = int(minStepsEpoch/numFiles)+1
if numSteps < minStepsImage:
    numSteps = minStepsImage
print('Steps per image:  '+str(numSteps))
print('Steps per epoch:  '+str(numSteps*numFiles))

newDirName=str(numSteps)+'_'+jobName
%cd $imageDir/images
!mv tmp $newDirName
print('Folder to caption:  '+imageDir+'/images/'+newDirName)

__Pause here__
- If you haven't started a new terminal session and ran ```tmux attach -t kohya_session```, do it now.
- Wait for the _kohya_ss_ UI to start, then click on the _gradio_ link.
- Run 'Utilities/WD14 Captioning'
- When captioning is finished, run the next cell.

In [ ]:
#Editing the config file
origConfigFile=configFile+'.orig'
!cp $origConfigFile $configFile
#!sed -i 's/\"output_name\": \"latest\",/\"output_name\": \"{jobName}\",/' $configFile
#!sed -i 's/\"max_resolution\": \"768,768\",/\"max_resolution\": \"{maxResolution}\",/' $configFile
#numEpochsStr=str(numEpochs)
#!sed -i 's/\"epoch\": \"10\",/\"epoch\": \"{numEpochsStr}\",/' $configFile
#enableBucketStr=str(enableBucket).lower()
#!sed -i 's/\"enable_bucket\": true,/\"enable_bucket\": {enableBucketStr},/' $configFile
#xformersStr=str(xformers).lower()
#!sed -i 's/\"xformers\": false,/\"xformers\": {xformersStr},/' $configFile
#print('Configuration file:  '+configFile)
for item in config.keys():
    if type(config[item]) is str:
        !sed -i 's/\"{item}\": .*?,/\"{item}\": \"{config[item]}\",/' $configFile
    else:
        value=str(config[item]).lower
        !sed -i 's/\"{item}\": .*?,/\"{item}\": {value},/' $configFile

__Almost done.__
- Load config file in 'Dreambooth LoRA'
- Set up sampling in 'Dreambooth LoRA/Sample images config'
- Start training in kohya_ss UI.